In [209]:
import pandas as pd
import numpy as np

### Import dataset

In [210]:
df = pd.read_csv("data/raw/michelin_by_Jerry_Ng.csv") 

### Exploratory data analysis

In [211]:
df.head(3)

,Name,Address,Location,Price,Cuisine,Longitude,Latitude,PhoneNumber,Url,WebsiteUrl,Award,GreenStar,FacilitiesAndServices,Description
0,Taïrroir,"6F, 299 Lequn 3rd Road, Zhongshan District, Ta...","Taipei, Taiwan",$$$$,Taiwanese contemporary,121.559303,25.082896,8.862850e+11,https://guide.michelin.com/en/taipei-region/ta...,https://www.tairroir.com/,3 Stars,0,"Air conditioning,Wheelchair access","A portmanteau of Taiwan and terroir, Taïrroir ..."
1,JL Studio,"2F, 689, Section 4, Yifeng Road, Nantun Distri...","Taichung, Taiwan",$$$$,"Singaporean, Contemporary",120.628520,24.150486,8.864238e+11,https://guide.michelin.com/en/taichung-region/...,https://jlstudiotw.com,3 Stars,0,"Air conditioning,Car park,Wheelchair access","JL stands for Jimmy Lim, a Singaporean chef wh..."
2,Le Palais,"17F, Palais de Chine Hotel, 3, Section 1, Chen...","Taipei, Taiwan",$$$$,Cantonese,121.516889,25.049163,8.862218e+11,https://guide.michelin.com/en/taipei-region/ta...,https://www.palaisdechinehotel.com/pdc-en/page...,3 Stars,0,"Air conditioning,Car park,Wheelchair access","After the departure of the Macanese chef, the ..."


In [212]:
df.shape

(15520, 14)

In [213]:
df.info()  # data type

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15520 entries, 0 to 15519
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Name                   15520 non-null  object 
 1   Address                15520 non-null  object 
 2   Location               15520 non-null  object 
 3   Price                  15519 non-null  object 
 4   Cuisine                15519 non-null  object 
 5   Longitude              15520 non-null  float64
 6   Latitude               15520 non-null  float64
 7   PhoneNumber            15182 non-null  float64
 8   Url                    15520 non-null  object 
 9   WebsiteUrl             13245 non-null  object 
 10  Award                  15520 non-null  object 
 11  GreenStar              15520 non-null  int64  
 12  FacilitiesAndServices  14665 non-null  object 
 13  Description            15519 non-null  object 
dtypes: float64(3), int64(1), object(10)
memory usage: 1.7+

In [214]:
df[
    "Award"
].value_counts()

Award
Selected Restaurants    10484
Bib Gourmand             3238
1 Star                   1160
2 Stars                   493
3 Stars                   145
Name: count, dtype: int64

In [215]:
df[
    "GreenStar"
].value_counts()

GreenStar
0    15279
1      241
Name: count, dtype: int64

In [216]:
df[
    "Name"
].nunique()

14980

In [217]:
df[
    "FacilitiesAndServices"
].nunique()

826

In [218]:
df[
    "Cuisine"
].value_counts().head(15)

Cuisine
Modern Cuisine           2425
Traditional Cuisine       760
Creative                  461
Japanese                  458
Italian                   404
Contemporary              400
Seafood                   353
French                    319
Street Food               308
Modern British            252
Mediterranean Cuisine     210
Classic Cuisine           206
Farm to table             195
Thai                      171
Cantonese                 165
Name: count, dtype: int64

In [219]:
df[
    "Price"
].value_counts()

Price
€€      4675
€€€     2773
€€€€    1278
$$      1155
$$$      838
$$$$     625
¥¥¥      573
€        553
$        495
££       451
¥¥       414
£££      301
฿฿       210
¥        198
¥¥¥¥     156
฿        128
££££     116
₩         87
₫         76
฿฿฿       71
₩₩        57
₫₫        48
₺₺        46
₩₩₩       32
₩₩₩₩      31
฿฿฿฿      29
₫₫₫       25
₺₺₺       20
₫₫₫₫      15
₺₺₺₺      15
£         14
₺         14
Name: count, dtype: int64

### Data cleaning

In [220]:
# Drop columns
df = df.drop(columns=["PhoneNumber", "Url", "WebsiteUrl", "Description"])

In [221]:
# Remove rows with NA values ​​only from columns "Price" and "Cuisine"
df = df.dropna(subset=["Price", "Cuisine"])

In [222]:
# Insert new column with price in dollars only
def convert_to_dollars(price):
    return "$" * len(price)

df["Price_dollar"] = df["Price"].apply(convert_to_dollars)

In [223]:
# Change 0 and 1 in the "GreenStar" column to Yes and No
df["GreenStar"] = df["GreenStar"].replace({0: "No", 1: "Yes"})

In [224]:
# Remove all cuisine types after comma
df["Cuisine"] = df["Cuisine"].str.split(",").str[0].str.strip()

In [225]:
df[
    "Cuisine"
].nunique()

259

In [226]:
# Normalize commas and return three items
def clean_facilities(phrase):
    if not isinstance(phrase, str):
        return phrase  # Return the original value if it's not a string

    # Normalize commas and split the phrase into items
    items = phrase.replace(",", ", ").split(", ")
    
    # Remove extra spaces and filter out empty items
    cleaned_items = [item.strip() for item in items if item.strip()]
    
    # Return the first three items formatted
    return ", ".join(cleaned_items[:3])

df["FacilitiesAndServices"] = df["FacilitiesAndServices"].apply(clean_facilities)

### Create columns

In [227]:
# Split "Location" column
df[['City', 'Country']] = df['Location'].str.split(',', n=1, expand=True).apply(lambda x: x.str.strip())

In [228]:
# Identify rows with NA
df[df['Country'].isna()][['City', 'Country']].drop_duplicates()


,City,Country
9,Singapore,None
67,Hong Kong,None
70,Macau,None
163,Dubai,None
3314,Luxembourg,None
3811,Abu Dhabi,None


In [229]:
location = {"Singapore": "Republic of Singapore",
              "Hong Kong": "Hong Kong SAR China",
              "Macau": "Macau SAR China",
              "Dubai": "United Arab Emirates",
              "Abu Dhabi": "United Arab Emirates",
              "Luxembourg": "Luxembourg",
              }

# Replace values ​​in the 'Country' column where they are NA
df['Country'] = df['Country'].fillna(df['City'].map(location))

In [230]:
# Replaces "Luxembourg" with NA in 'City' where 'City' and 'Country' are both "Luxembourg"
df.loc[df[['City', 'Country']].eq("Luxembourg").all(axis=1), 'City'] = np.nan


In [231]:
# Create more detailed information for the column
award_info_en = {"3 Stars": "Exceptional cuisine",
              "2 Stars": "Excellent cooking",
              "1 Star": "High quality cooking",
              "Bib Gourmand": "Good quality, good value cooking",
              "Selected Restaurants": "Good cooking",
              }

df["Award_info_en"] = df["Award"].map(award_info_en)

In [232]:
# Create more detailed information for the column and create translation
award_info_br = {"3 Stars": "Cozinha excepcional",
              "2 Stars": "Cozinha excelente",
              "1 Star": "Cozinha requintada",
              "Bib Gourmand": "Melhor relação qualidade-preço ",
              "Selected Restaurants": "Boa cozinha",
              }

df["Award_info_br"] = df["Award"].map(award_info_br)

### Translate columns

In [233]:
from googletrans import Translator
import asyncio

In [234]:
translator = Translator()

async def translate_text(text, src="en", dest="pt"):
    try:
        translated = (await translator.translate(text, src=src, dest=dest)).text
        return translated.capitalize()
    except Exception:
        return text.capitalize()

async def translate_column(column):
    unique_values = column.unique()
    translations = await asyncio.gather(*(translate_text(text) for text in unique_values))
    translation_dict = dict(zip(unique_values, translations))
    return column.map(translation_dict)

#### FacilitiesAndServices

In [235]:
df["FacilitiesAndServices_br"] = await translate_column(df["FacilitiesAndServices"])

#### Country and Location

In [236]:
unique_award = df["Country"].unique()
print(unique_award)

['Taiwan' 'USA' 'Republic of Singapore' 'Slovenia' 'Sweden' 'Denmark'
 'Norway' 'Japan' 'Germany' 'France' 'Hong Kong SAR China'
 'Macau SAR China' 'Belgium' 'United Kingdom' 'Spain' 'China Mainland'
 'Italy' 'Switzerland' 'Netherlands' 'Austria' 'United Arab Emirates'
 'Croatia' 'Poland' 'Finland' 'Estonia' 'Brazil' 'Mexico' 'Malta'
 'Portugal' 'Luxembourg' 'South Korea' 'Ireland' 'Thailand' 'Argentina'
 'Malaysia' 'Türkiye' 'Greece' 'Canada' 'Hungary' 'Vietnam' 'Lithuania'
 'Iceland' 'Serbia' 'Latvia' 'Czech Republic' 'Czechia' 'Andorra']


In [237]:
df["Country_br"] = await translate_column(df["Country"])

In [238]:
# Create column "Location_br"
df["Location_br"] = df["City"] + ", " + df["Country_br"]

In [239]:
# Check missing values
df[df['Location_br'].isna()][['Location', 'Location_br']].drop_duplicates()


,Location,Location_br
3314,Luxembourg,NaN


In [240]:
# Manually translate missing values
translation_map = {
    'Luxembourg': 'Luxemburgo',
}

df['Location_br'] = df.apply(
    lambda row: translation_map.get(row['Location'], row['Location_br']), axis=1
)

#### Cuisine

In [241]:
unique_cuisines = df["Cuisine"].unique()
print(unique_cuisines)

['Taiwanese contemporary' 'Singaporean' 'Cantonese' 'Contemporary' 'Asian'
 'European Contemporary' 'French Contemporary' 'French' 'Creative'
 'Modern Cuisine' 'Japanese' 'Innovative' 'Classic French' 'Modern French'
 'Seafood' 'Classic Cuisine' 'Mediterranean Cuisine' 'Italian' 'Sushi'
 'Creative French' 'Creative British' 'Modern British' 'Chinese'
 'Italian Contemporary' 'American' 'Vegan' 'Chao Zhou' 'Taizhou'
 'Vegetarian' 'Spanish Contemporary' 'Asian Contemporary' 'Tempura'
 'Mexican' 'Californian' 'Indian' 'British Contemporary' 'Scandinavian'
 'Hunanese' 'Huaiyang' 'Traditional Cuisine' 'Asian Influences' 'Korean'
 'Turkish' 'Traditional British' 'Thai contemporary' 'German'
 'Southern Thai' 'Thai' 'Spanish' 'Malaysian' 'Country cooking'
 'International' 'Piedmontese' 'Beijing Cuisine' 'Shanghainese' 'Sharing'
 'Farm to table' 'Japanese Contemporary' 'Sichuan' 'Organic'
 'Latin American' 'Taiwanese' 'Hang Zhou' 'Barbecue' 'Steakhouse'
 'Portuguese' 'Indian Vegetarian' 'Vietnam

In [242]:
df["Cuisine_br"] = await translate_column(df["Cuisine"])

#### Price

In [243]:
unique_award = df["Award"].unique()
print(unique_award)

['3 Stars' '2 Stars' '1 Star' 'Bib Gourmand' 'Selected Restaurants']


In [244]:
df["Award_br"] = await translate_column(df["Award"])

### Add scraped images

In [245]:
df_scrap = pd.read_csv("data/scraper/images_url.csv")

In [246]:
df_final = pd.merge(df, df_scrap, on="Address", suffixes=("", "_drop")).drop(columns=["Name_drop"])

### Save dataset

In [247]:
# Drop columns
df_final = df_final.drop(columns=["City", "Country", "Country_br"])

In [248]:
# Convert all columns in the list to type "category"
columns_category = ["Location", "Location_br", "Price", "Price_dollar", "Cuisine", "Cuisine_br", "Award", "Award_br", "FacilitiesAndServices", "FacilitiesAndServices_br"]

for column in columns_category:
    df_final[column] = df_final[column].astype("category")

In [249]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15498 entries, 0 to 15497
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   Name                      15498 non-null  object  
 1   Address                   15498 non-null  object  
 2   Location                  15498 non-null  category
 3   Price                     15498 non-null  category
 4   Cuisine                   15498 non-null  category
 5   Longitude                 15498 non-null  float64 
 6   Latitude                  15498 non-null  float64 
 7   Award                     15498 non-null  category
 8   GreenStar                 15498 non-null  object  
 9   FacilitiesAndServices     14662 non-null  category
 10  Price_dollar              15498 non-null  category
 11  Award_info_en             15498 non-null  object  
 12  Award_info_br             15498 non-null  object  
 13  FacilitiesAndServices_br  15498 non-null  cate

In [250]:
df_final.to_parquet("data/ready/michelin.parquet")

In [251]:
df_final.head(3)

,Name,Address,Location,Price,Cuisine,Longitude,Latitude,Award,GreenStar,FacilitiesAndServices,Price_dollar,Award_info_en,Award_info_br,FacilitiesAndServices_br,Location_br,Cuisine_br,Award_br,ImageURL
0,Taïrroir,"6F, 299 Lequn 3rd Road, Zhongshan District, Ta...","Taipei, Taiwan",$$$$,Taiwanese contemporary,121.559303,25.082896,3 Stars,No,"Air conditioning, Wheelchair access",$$$$,Exceptional cuisine,Cozinha excepcional,"Ar condicionado, acesso a cadeira de rodas","Taipei, Taiwan",Contemporâneo de taiwan,3 estrelas,https://axwwgrkdco.cloudimg.io/v7/__gmpics3__/...
1,JL Studio,"2F, 689, Section 4, Yifeng Road, Nantun Distri...","Taichung, Taiwan",$$$$,Singaporean,120.628520,24.150486,3 Stars,No,"Air conditioning, Car park, Wheelchair access",$$$$,Exceptional cuisine,Cozinha excepcional,"Ar condicionado, estacionamento, acesso a cade...","Taichung, Taiwan",Cingapura,3 estrelas,https://axwwgrkdco.cloudimg.io/v7/__gmpics3__/...
2,Le Palais,"17F, Palais de Chine Hotel, 3, Section 1, Chen...","Taipei, Taiwan",$$$$,Cantonese,121.516889,25.049163,3 Stars,No,"Air conditioning, Car park, Wheelchair access",$$$$,Exceptional cuisine,Cozinha excepcional,"Ar condicionado, estacionamento, acesso a cade...","Taipei, Taiwan",Cantonês,3 estrelas,https://axwwgrkdco.cloudimg.io/v7/__gmpics3__/...
